# Tradutor Azure

Este notebook será composto pela implementação de duas funções com a finalidade de colocar em prática conteúdos sobre as ferramentas de processamento de linguagem natural da Azure.

Primeiramente será implementada a funcionalidade de tradução, onde um documento Word(.docx) em inglês servirá de entrada para que, utilizando serviço azure, seja traduzido para o português-br.

Outrossim, será testado, além da tradução, a extração de um texto da internet e o formantando via markdown apartir de um chatbox usando gpt4 mini.


### Traduzir o Texto

In [1]:
import requests
from docx import Document 
import uuid

In [2]:
endpoint = "https://api.cognitive.microsofttranslator.com/"
path = "/translate"
constructed_url = endpoint + path

In [3]:
autentication_key = "FOjxid7pey3CitXQaukXcAYwem4fu24uNp5GDKIAyPPi4APcv6hAJQQJ99AKACYeBjFXJ3w3AAAbACOGLE6A"
location = "eastus"
header = {
   'Ocp-Apim-Subscription-Key': autentication_key,
   'Ocp-Apim-Subscription-Region': location,
   'Content-type': 'application/json',
   'X-ClientTraceId': str(uuid.uuid4())
}

In [4]:
params = {
   'api-version': '3.0',
   'from': 'en',
   'to': 'pt-br'
}

In [5]:
def translate_text(text):
   body = [{
      'text': text
   }]
   request = requests.post(url=constructed_url,params=params,headers=header,json=body)
   response = request.json()

   return response

In [6]:
mensage = "For this reason, embedded development is considered a domain on its own in the universe of software development, in which the developer’s approach and workflow need to be adapted accordingly."

In [7]:
translated_text = translate_text(mensage)
print(translated_text)

[{'translations': [{'text': 'Por esse motivo, o desenvolvimento embarcado é considerado um domínio por si só no universo do desenvolvimento de software, no qual a abordagem e o fluxo de trabalho do desenvolvedor precisam ser adaptados de acordo.', 'to': 'pt'}]}]


## Traduzir Documento

In [8]:
doc_path = "D:\Cursos\DIO\IA102\PrefaceEmbeddedSistemsArchitecture.docx"

In [9]:
def translate_doc(path):
   
   document = Document(path)
   translated_document = []
   for paragraph in document.paragraphs:
      translated_paragraph = translate_text(paragraph.text)
      translated_document.append(translated_paragraph)
   
   new_doc = Document()
   for line in translated_document:
      new_doc.add_paragraph(line[0]['translations'][0]['text'])
   path_translated = path.replace(".docx","_pt_br.docx")
   new_doc.save(path_translated)
   
   print("Tradução Completa")

In [10]:
translate_doc(doc_path)

Tradução Completa


## Reescrever Markdown

In [11]:
from bs4 import BeautifulSoup
from langchain_openai.chat_models.azure import AzureChatOpenAI

### Web Scraping

In [12]:
url = "https://dev.to/rose_michelle/beginner-friendly-python-projects-with-source-code-234e"

In [13]:
def extract_text_from_url(url):
   response = requests.get(url)
   soup = BeautifulSoup(response.text,'html.parser')

   for destroy in soup(['script','style']):
      destroy.decompose()
   
   texto = soup.get_text(separator=" ")
   linhas = (line.strip() for line in texto.splitlines())
   parts = (phrase.strip() for line in linhas for phrase in line.split("  "))
   texto_limpo = '\n'.join(part for part in parts if part)

   return texto_limpo
   

In [14]:
texto = extract_text_from_url(url)

In [15]:
texto

'Beginner-Friendly Python Projects with Source Code - DEV Community\nSkip to content\nNavigation menu\nSearch\nPowered by\nSearch\nAlgolia\nSearch\nLog in\nCreate account\nDEV Community\nClose\nAdd reaction\nLike\nUnicorn\nExploding Head\nRaised Hands\nFire\nJump to Comments\nSave\nBoost\nMore...\nCopy link\nCopy link\nCopied to Clipboard\nShare to X\nShare to LinkedIn\nShare to Facebook\nShare to Mastodon\nReport Abuse\nNamubiru Rose\nPosted on\nNov 10\nBeginner-Friendly Python Projects with Source Code\n# webdev\n# beginners\n# python\n# programming\nIntroduction\nStarting with beginner-friendly Python projects is an excellent way to solidify your understanding of coding fundamentals. As you work on these small projects, you’ll improve essential skills, including working with data types, managing user inputs, using conditionals, and handling basic logic. These projects are designed to be accessible to those new to programming and will help you practice Python concepts in a practical 

### Traduzir via Azure OpenAI

In [23]:
client = AzureChatOpenAI(
   azure_endpoint="https://laboratoriochatapp.openai.azure.com/",
   api_key="979nUCFkX0tiT8qSiY5KhKiavpuTepGf54xVHgXLBNCq26LEcVtPJQQJ99AKACYeBjFXJ3w3AAABACOGSYP4",
   api_version="2024-02-15-preview",
   deployment_name = "gpt-4o-mini",
   max_tokens=None,
   max_retries=0
)

In [19]:
def translate_document(text, client):
   message = [("system", "Você atua como tradutor de texto"),
              ("user",f"Traduza o texto {text} e o edite em Markdown")]
   response = client.invoke(message)
   return response.content


In [24]:
traduzido = translate_document(texto, client)

In [25]:
traduzido

'**Projetos de Python para Iniciantes com Código Fonte - DEV Community**\n\n**Introdução**\nComeçar com projetos de Python amigáveis para iniciantes é uma excelente maneira de solidificar sua compreensão dos fundamentos da programação. Ao trabalhar nesses pequenos projetos, você melhorará habilidades essenciais, incluindo trabalho com tipos de dados, gerenciamento de entradas do usuário, uso de condicionais e manipulação de lógica básica. Esses projetos são projetados para serem acessíveis àqueles que são novos na programação e ajudarão você a praticar conceitos de Python de maneira prática. Abaixo, apresentamos cinco projetos populares de Python, completos com guias passo a passo e exemplos de código.\n\n**1. Calculadora Básica**\n*Por que este projeto?*\nUma calculadora é um projeto fundamental que combina entrada do usuário, definições de função e aritmética básica. É perfeito para iniciantes, pois ensina conceitos centrais como o uso de funções e manipulação básica de erros (por ex